In [309]:
# Importing libraries
import numpy as np
import pandas as pd

In [310]:
# Getting the data from my github

# Imports
from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile

# Reading in the url
zipurl = 'https://github.com/silver-storm/machine_learning_codes/blob/master/Datasets/house-prices-advanced-regression-techniques.zip?raw=true'
with urlopen(zipurl) as ret:
    with ZipFile(BytesIO(ret.read())) as zipfile:
        """ Extracting and storing the data """
        zipfile.extractall('tmp/')

In [311]:
import os
os.listdir('tmp/')

['test.csv', 'sample_submission.csv', 'data_description.txt', 'train.csv']

In [312]:
# Reading in the Dataset

dataset = pd.read_csv('tmp/train.csv')
dataset.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [313]:
""" Uncomment to check Data Description """

# print("DATA DESCRIPTION\n\n")
# with open('../tmp/data_description.txt','r') as f:
#   for line in f.readlines():
#     print(line)

' Uncomment to check Data Description '

In [314]:
# Maintaining a copy of the original dataset for resetting in the case of irreversible erronneous changes.
# This is fine since this is a small dataset. Not recommended for large datasets.
df = dataset.copy()

In [315]:
df.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [316]:
def onehot_enc(data,col):
    """
        Function to generate one-hot representation of a column. 

        INPUT : Takes in a pandas DataFrame and a column name.
        OUTPUT : Returns the DataFrame with one_hot_vectors after adjusting for dummy variable trap.

        NOTE : This function replaces the original column with dummy vectors.
    """
    if col not in data.columns: 
      print('Please check column name');
      return data
    # drop_first = True to escape the dummy variable trap
    dummy_df = pd.get_dummies(data[col],prefix=col,drop_first=True)
    # Setting column names for the one-hot vectors ...
    dummy_df.columns = [elem.rstrip().split(' ')[0] for elem in dummy_df.columns]
    # Modifying the dataset and removing original column ...
    data = pd.concat([data.drop(col, axis=1),dummy_df],axis=1)
    return data

In [317]:
def label_enc(data,col):
  """
     Function to label encode the required column.

     INPUT : Takes in a pandas DataFrame and a column name.
     OUTPUT : Returns the DataFrame with the label encoded column
     
     NOTE : This function replaces the original column with labels.
  """
  if col not in data.columns: 
      print('Please check column name');
      return data
  if col in ['Alley','Street']:
    data[col] = data[col].map({'Grvl':1,'Pave':2,'None':0})
  elif col == 'LotShape':
    data[col] = data[col].map({'Reg':3,'IR1':2,'IR2':1,'IR3':0})
  elif col == 'LandContour':
    data[col] = data[col].map({'Lvl':3,'Bnk':2,'HLS':1,'Low':0})
  elif col == 'Utilities':
    data[col] = data[col].map({'AllPub':3,'NoSewr':2,'NoSeWa':1,'ELO':0})
  elif col == 'LotConfig':
    data[col] = data[col].map({'Inside':4,'Corner':3,'CulDSac':2,'FR2':1,'FR3':0})
  elif col == 'LandSlope':
    data[col] = data[col].map({'Gtl':2,'Mod':1,'Sev':0})
  elif col == 'Functional':
    data[col] = data[col].map({'Typ':7,'Min1':6,'Min2':5,'Mod':4,'Maj1':3,'Maj2':2,'Sev':1,'Sal':0})
  elif col == 'GarageType':
    data[col] = data[col].map({'2Types':5,'Attchd':3,'Basment':3,'BuiltIn':3,'CarPort':2,'Detchd':1,'None':0})
  elif col == 'GarageFinish':
    data[col] = data[col].map({'Fin':3,'RFn':2,'Unf':1,'None':0})
  elif col == 'Fence':
      data[col] = data[col].map({'GdPrv':4,'MnPrv':2,'GdWo':3,'MnWw':1,'None':0})
  elif col in ['ExterQual','ExterCond','BsmtQual','BsmtCond','KitchenQual','FireplaceQu','GarageQual','GarageCond','HeatingQC','PoolQC']:
    data[col] = data[col].map({'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,'None':0})
  elif col in ['CentralAir','PavedDrive']:
    data[col] = data[col].map({'N':0,'P':1,'Y':2})
  elif col == 'BsmtExposure':
    data[col] = data[col].map({'Gd':4,'Av':3,'Mn':2,'No':1,'None':0})
  elif col in ['BsmtFinType1','BsmtFinType2']:
    data[col] = data[col].map({'GLQ':6,'ALQ':5,'BLQ':4,'Rec':3,'LwQ':2,'Unf':1,'None':0})
  return data

In [318]:
# To identify which columns are numerical, string and contain nan's.
# This helps tremendously during data preprocessing.

numeric_cols = []
numeric_nan_cols=[]
string_cols = []
for col in df.columns:
    if pd.api.types.is_numeric_dtype(df[col]):
        numeric_cols.append(col)
        if sum(np.multiply(df[col].isnull(),1)):
            numeric_nan_cols.append(col)
    if pd.api.types.is_string_dtype(df[col]):
        string_cols.append(col)
        df[col].fillna('None',inplace=True)

In [319]:
# Printing out the columns with NaN values ...
numeric_nan_cols

['LotFrontage', 'MasVnrArea', 'GarageYrBlt']

In [320]:
# Replacing the NaNs with suitable values ...
df['LotFrontage'].fillna(0,inplace=True)
df['LotFrontage'] = df['LotFrontage'].astype('int')
df['MasVnrArea'].fillna(0,inplace=True)
df['MasVnrArea'] = df['MasVnrArea'].astype('int')
df['GarageYrBlt'].fillna(1900.0,inplace=True)
df['GarageYrBlt'] = df['GarageYrBlt'].astype('int')

In [141]:
#df.drop(['LowQualFinSF','MiscFeature'],axis=1,inplace=True)

In [321]:
# Storing the columns to be encoded ...
label_enc_cols = ['Street','Alley','LotShape','LandContour','Utilities','LotConfig','LandSlope','ExterQual','ExterCond','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1',
                  'BsmtFinType2','HeatingQC','CentralAir','Functional','KitchenQual','FireplaceQu','GarageFinish','GarageQual',
                  'GarageCond','PavedDrive','PoolQC','Fence']
one_hot_cols = [col for col in np.setdiff1d(string_cols,label_enc_cols)]

In [322]:
# Encoding the required columns
for col in label_enc_cols:
    df = label_enc(df,col)
for col in one_hot_cols:
    df = onehot_enc(df,col)

In [324]:
print("Number of features before preprocessing : ",len(dataset.columns),"\nNumber of features after preprocessing : ",len(df.columns))

Number of features before preprocessing :  81 
Number of features after preprocessing :  199


In [325]:
# Checking if all data types in final preprocessed dataset is numeric or not

for i,typ in enumerate(df.dtypes):
    if typ not in ['int64','uint8']:
      print(typ,df.columns[i])

In [326]:
def rem_Redundant_Features(data):
    cor_mat = df.corr().abs()
    upper_tri = cor_mat.where(np.triu(np.ones(cor_mat.shape),k=1).astype(np.bool))
    to_drop = [col for col in upper_tri.columns if any(upper_tri[col] > 0.95)]
    print("Dropping redundant features : \n")
    for col in to_drop:
      print(col)
    df.drop(to_drop, axis=1, inplace=True)
    return df

In [290]:
#df = rem_Redundant_Features(df)

Dropping redundant features : 

GarageCond
Exterior2nd_CBlock
Exterior2nd_CmentBd
Exterior2nd_MetalSd
Exterior2nd_VinylSd
Exterior2nd_Wd
Exterior2nd_Wd
MiscFeature_Shed
SaleType_New


In [327]:
y = df['SalePrice'].values
df = df.drop(columns=['SalePrice'],axis=1)
X = df.iloc[:,1:].values

In [328]:
y.shape

(1460,)

In [329]:
X.shape

(1460, 197)

In [330]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import MinMaxScaler
sc_X = MinMaxScaler(feature_range=(0, 1))
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)
sc_y = MinMaxScaler(feature_range=(0, 1))
y_train = sc_y.fit_transform(y_train.reshape(-1,1))

In [331]:
X_train.shape

(1314, 197)

In [332]:
y_train.shape

(1314, 1)

In [333]:
def Evaluate(regressor,regr_name):
    # Predicting the Test set results
    y_pred = regressor.predict(X_test)
    y_pred = sc_y.inverse_transform(y_pred.reshape(-1,1))
    
    from sklearn.metrics import mean_squared_error,r2_score
    print("The RMSE of the "+regr_name+" model is : ",np.sqrt(mean_squared_error(y_test,y_pred)))
    print("The R2 Score of the "+regr_name+" model is : ",r2_score(y_test,y_pred))

## LINEAR REGRESSION

In [334]:
# Fitting Multiple Linear Regression to the Training set
from sklearn.linear_model import LinearRegression
regressor_lm = LinearRegression().fit(X_train, y_train)

Evaluate(regressor_lm,"Linear Regression")

The RMSE of the Linear Regression model is :  83273663783367.38
The R2 Score of the Linear Regression model is :  -1.0090783973570217e+18


## DECISION TREE REGRESSION

In [335]:
# Fitting Decision Tree Regression to the dataset
from sklearn.tree import DecisionTreeRegressor
dt_regressor = DecisionTreeRegressor(random_state = 42).fit(X_train, y_train)

Evaluate(dt_regressor,"Decision Tree Regression")

The RMSE of the Decision Tree Regression model is :  48002.23014534189
The R2 Score of the Decision Tree Regression model is :  0.6647008958940683


## RANDOM FOREST REGRESSION

In [336]:
# Fitting Random Forest Regression to the dataset
from sklearn.ensemble import RandomForestRegressor
rf_regressor = RandomForestRegressor(n_estimators = 15, random_state = 52).fit(X_train, y_train.ravel())

Evaluate(rf_regressor,"Random Forest Regression")

The RMSE of the Random Forest Regression model is :  28774.221004089297
The R2 Score of the Random Forest Regression model is :  0.8795195133002778


## SUPPORT VECTOR REGRESION

In [337]:
# Fitting SVR to the dataset
from sklearn.svm import SVR
svm_regressor = SVR(kernel = 'rbf').fit(X_train, y_train.ravel())

Evaluate(svm_regressor,"Support Vector Regression")

The RMSE of the Support Vector Regression model is :  47935.96774981091
The R2 Score of the Support Vector Regression model is :  0.6656259523796322


## HUBER REGRESSION

In [339]:
import warnings
warnings.filterwarnings("ignore")

# Fitting Huber Regression to dataset

from sklearn.linear_model import HuberRegressor
hb_regressor = HuberRegressor(max_iter=200).fit(X_train,y_train.ravel())

Evaluate(hb_regressor,"Huber Regression")

The RMSE of the Huber Regression model is :  32409.807342745058
The R2 Score of the Huber Regression model is :  0.84715104910501


## ADAPTIVE BOOSTING

In [340]:
from sklearn.ensemble import AdaBoostRegressor
adb_regressor = AdaBoostRegressor().fit(X_train,y_train.ravel())

Evaluate(adb_regressor,"Adaboost Regression")

The RMSE of the Adaboost Regression model is :  38896.28857257875
The R2 Score of the Adaboost Regression model is :  0.7798463498334485


## XGBOOSTING

In [341]:
import xgboost as xgb
xg_reg = xgb.XGBRegressor(objective='reg:squarederror',n_estimators=1000).fit(X_train,y_train)

Evaluate(xg_reg,"XGBoost Regression")

The RMSE of the XGBoost Regression model is :  32990.08470108905
The R2 Score of the XGBoost Regression model is :  0.8416287208249675


## DEEP NEURAL NETWORK

In [342]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Dropout

In [343]:
# Get input shapes
N, D = X_train.shape

In [344]:
N,D

(1314, 197)

In [354]:
I = Input(shape=(D,))
x = Dense(D, activation='relu')(I)
x = Dense(D, activation='relu')(x)
x = Dropout(0.3,seed=42)(x)
x = Dense(1, activation='sigmoid')(x)
model = Model(inputs=I, outputs=x)

model.compile(loss='mse',optimizer='adam',)

r = model.fit(X_train, y_train, epochs=40,verbose=0)
Evaluate(model,"Deep Neural Network")

The RMSE of the Deep Neural Network model is :  32681.64769257582
The R2 Score of the Deep Neural Network model is :  0.8445762256089361
